In [1]:
import os
print(os.getcwd())

C:\Users\Jaya Prakash\Desktop\ADV ML\Data


In [2]:
import numpy as np
import pandas as pd

In [3]:
data = pd.read_csv("./NER_Dataset.csv")

In [4]:
data.describe()

,Sentence_ID,Word,POS,Tag
count,47959,47959,47959,47959
unique,47959,47575,47214,33318
top,Sentence: 1,"['VOA', ""'s"", 'Mil', 'Arcega', 'reports', '.']","['NNP', 'POS', 'NNP', 'NNP', 'VBZ', '.']","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
freq,1,17,39,450


In [5]:
print(data.columns)

Index(['Sentence_ID', 'Word', 'POS', 'Tag'], dtype='object')


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47959 entries, 0 to 47958
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Sentence_ID  47959 non-null  object
 1   Word         47959 non-null  object
 2   POS          47959 non-null  object
 3   Tag          47959 non-null  object
dtypes: object(4)
memory usage: 1.5+ MB


In [7]:
data.head(10)

,Sentence_ID,Word,POS,Tag
0,Sentence: 1,"['Thousands', 'of', 'demonstrators', 'have', '...","['NNS', 'IN', 'NNS', 'VBP', 'VBN', 'IN', 'NNP'...","['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', '..."
1,Sentence: 10,"['Iranian', 'officials', 'say', 'they', 'expec...","['JJ', 'NNS', 'VBP', 'PRP', 'VBP', 'TO', 'VB',...","['B-gpe', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '..."
2,Sentence: 100,"['Helicopter', 'gunships', 'Saturday', 'pounde...","['NN', 'NNS', 'NNP', 'VBD', 'JJ', 'NNS', 'IN',...","['O', 'O', 'B-tim', 'O', 'O', 'O', 'O', 'O', '..."
3,Sentence: 1000,"['They', 'left', 'after', 'a', 'tense', 'hour-...","['PRP', 'VBD', 'IN', 'DT', 'NN', 'JJ', 'NN', '...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
4,Sentence: 10000,"['U.N.', 'relief', 'coordinator', 'Jan', 'Egel...","['NNP', 'NN', 'NN', 'NNP', 'NNP', 'VBD', 'NNP'...","['B-geo', 'O', 'O', 'B-per', 'I-per', 'O', 'B-..."
5,Sentence: 10001,"['Mr.', 'Egeland', 'said', 'the', 'latest', 'f...","['NNP', 'NNP', 'VBD', 'DT', 'JJS', 'NNS', 'VBP...","['B-per', 'I-per', 'O', 'O', 'O', 'O', 'O', 'O..."
6,Sentence: 10002,"['He', 'said', 'last', 'week', ""'s"", 'tsunami'...","['PRP', 'VBD', 'JJ', 'NN', 'POS', 'NN', 'CC', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
7,Sentence: 10003,"['Some', '1,27,000', 'people', 'are', 'known',...","['DT', 'CD', 'NNS', 'VBP', 'VBN', 'JJ', '.']","['O', 'O', 'O', 'O', 'O', 'O', 'O']"
8,Sentence: 10004,"['Aid', 'is', 'being', 'rushed', 'to', 'the', ...","['NNP', 'VBZ', 'VBG', 'VBN', 'TO', 'DT', 'NN',...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
9,Sentence: 10005,"['Lebanese', 'politicians', 'are', 'condemning...","['JJ', 'NNS', 'VBP', 'VBG', 'NNP', 'POS', 'NN'...","['B-gpe', 'O', 'O', 'O', 'B-tim', 'O', 'O', 'O..."


In [8]:
from itertools import chain
def get_dict_map(data, token_or_tag):
    tok2idx = {}
    idx2tok = {}
    
    if token_or_tag == 'token':
        vocab = list(set(data['Word'].to_list()))
    else:
        vocab = list(set(data['Tag'].to_list()))
    
    idx2tok = {idx:tok for  idx, tok in enumerate(vocab)}
    tok2idx = {tok:idx for  idx, tok in enumerate(vocab)}
    return tok2idx, idx2tok
token2idx, idx2token = get_dict_map(data, 'token')
tag2idx, idx2tag = get_dict_map(data, 'tag')

In [9]:
data['Word_idx'] = data['Word'].map(token2idx)
data['Tag_idx'] = data['Tag'].map(tag2idx)
data_fillna = data.ffill(axis=0)  # Forward fill missing values

# Groupby and collect columns
data_group = data_fillna.groupby(
['Sentence_ID'],as_index=False
)[['Word', 'POS', 'Tag', 'Word_idx', 'Tag_idx']].agg(lambda x: list(x))

In [10]:
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

def get_pad_train_test_val(data_group, data):

    #get max token and tag length
    n_token = len(list(set(data['Word'].to_list())))
    n_tag = len(list(set(data['Tag'].to_list())))

    #Pad tokens (X var)    
    tokens = data_group['Word_idx'].tolist()
    maxlen = max([len(s) for s in tokens])
    pad_tokens = pad_sequences(tokens, maxlen=maxlen, dtype='int32', padding='post', value= n_token - 1)
    
    #Pad Tags (y var) and convert it into one hot encoding
    tag2idx['O'] = len(tag2idx)  # Assign the next available index value to 'O'
    tags = data_group['Tag_idx'].tolist()
    pad_tags = pad_sequences(tags, maxlen=maxlen, dtype='int32', padding='post', value= tag2idx["O"])
    n_tags = len(tag2idx)
    pad_tags = [to_categorical(i, num_classes=n_tags) for i in pad_tags]
    
    #Split train, test and validation set
    tokens_, test_tokens, tags_, test_tags = train_test_split(pad_tokens, pad_tags, test_size=0.1, train_size=0.9, random_state=2020)
    train_tokens, val_tokens, train_tags, val_tags = train_test_split(tokens_,tags_,test_size = 0.25,train_size =0.75, random_state=2020)

    print(
        'train_tokens length:', len(train_tokens),
        '\ntrain_tokens length:', len(train_tokens),
        '\ntest_tokens length:', len(test_tokens),
        '\ntest_tags:', len(test_tags),
        '\nval_tokens:', len(val_tokens),
        '\nval_tags:', len(val_tags),
    )
    
    return train_tokens, val_tokens, test_tokens, train_tags, val_tags, test_tags

train_tokens, val_tokens, test_tokens, train_tags, val_tags, test_tags = get_pad_train_test_val(data_group, data)


train_tokens length: 32372 
train_tokens length: 32372 
test_tokens length: 4796 
test_tags: 4796 
val_tokens: 10791 
val_tags: 10791


In [11]:
import numpy as np
import tensorflow
from tensorflow.keras import Sequential, Model, Input
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from tensorflow.keras.utils import plot_model
from numpy.random import seed
seed(1)
tensorflow.random.set_seed(2)


In [12]:
input_dim = len(list(set(data['Word'].to_list())))+1
output_dim = 64
input_length = max([len(s) for s in data_group['Word_idx'].tolist()])
n_tags = len(tag2idx)

In [13]:
def get_bilstm_lstm_model():
    model = Sequential()

    # Add Embedding layer
    model.add(Embedding(input_dim=input_dim, output_dim=output_dim, input_length=input_length))

    # Add bidirectional LSTM
    model.add(Bidirectional(LSTM(units=output_dim, return_sequences=True, dropout=0.2, recurrent_dropout=0.2), merge_mode = 'concat'))

    # Add LSTM
    model.add(LSTM(units=output_dim, return_sequences=True, dropout=0.5, recurrent_dropout=0.5))

    # Add timeDistributed Layer
    model.add(TimeDistributed(Dense(n_tags, activation="relu")))

    #Optimiser 
    # adam = k.optimizers.Adam(lr=0.0005, beta_1=0.9, beta_2=0.999)

    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    
    return model
def train_model(X, y, model):
    loss = list()
    for i in range(25):
        # fit model for one epoch on this sequence
        hist = model.fit(X, y, batch_size=1000, verbose=1, epochs=1, validation_split=0.2)
        loss.append(hist.history['loss'][0])
    return loss
from keras_visualizer import visualizer
model_bilstm_lstm = get_bilstm_lstm_model()
visualizer(model_bilstm_lstm)
results = pd.DataFrame()
results['with_add_lstm'] = train_model(train_tokens, np.array(train_tags), model_bilstm_lstm)
model_bilstm_lstm.save('model_bilstm_lstm.h5')  # Saves the model to a HDF5 file


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 1, 64)             3044864   
                                                                 
 bidirectional_2 (Bidirecti  (None, 1, 128)            66048     
 onal)                                                           
                                                                 
 lstm_5 (LSTM)               (None, 1, 64)             49408     
                                                                 
 time_distributed_2 (TimeDi  (None, 1, 33319)          2165735   
 stributed)                                                      
                                                                 
Total params: 5326055 (20.32 MB)
Trainable params: 5326055 (20.32 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
26/26 [=======

C:\Users\Jaya Prakash\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [34]:
import spacy
from spacy import displacy
from tensorflow.keras.models import load_model
# Load the model
model = load_model('model_bilstm_lstm.h5')
text = model('Hi, My name is Aman Kharwal \n I am from India \n I want to work with Google \n Steve Jobs is My Inspiration')
displacy.render(text, style = 'ent', jupyter=True)

TypeError: Inputs to a layer should be tensors. Got 'Hi, My name is Aman Kharwal 
 I am from India 
 I want to work with Google 
 Steve Jobs is My Inspiration' (of type <class 'str'>) as input for layer 'sequential_2'.